In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Model=VGG16,
# Optimizer=NAdam,
# Learning rate= .00001,
# Used downsampled dataset

In [ ]:
import os
import glob
import shutil
import json
import keras
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import unicodedata
import matplotlib.pyplot as plt
import seaborn as sns
import os, re
from pprint import pprint
import matplotlib.patches as patches
from matplotlib.cbook import print_cycles
from PIL import Image
import glob
import cv2
from skimage import color
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop, Adam
import numpy as np
import os
from pathlib import Path

In [ ]:
base_dir = Path('/kaggle/input/cassava-leaf-disease-classification/')
data  = pd.read_csv(f'{base_dir}/train.csv')

In [ ]:
data2=data.copy()
data2.head()

# **Data preprocessing**

In [ ]:
label_dict = {
0:"Cassava Bacterial Blight (CBB)",
1:"Cassava Brown Streak Disease (CBSD)",
2:"Cassava Green Mottle (CGM)",
3:"Cassava Mosaic Disease (CMD)",
4:"Healthy"
}
label_dict

In [ ]:
data['label'].replace(label_dict, inplace=True)

In [ ]:
data.head()

# **Downsampling**

In [ ]:
mask = data2['label'] ==4
classHealthy = data[mask]
mask = data2['label']==3
classCMD = data[mask]
mask = data2['label'] ==2
classCGM  = data[mask]
mask = data2['label'] ==1
classCBSD  = data[mask]
mask = data2['label'] ==0
classCBB  = data[mask]

In [ ]:
data2['label'].hist()

In [ ]:
class0 = classCBB.sample(frac=0.99)
class1 = classCBSD.sample(frac=0.99)
class2 = classCGM.sample(frac=0.99)
class3 = classCMD.sample(frac=0.5)
class4 = classHealthy.sample(frac=0.99)
frames=[class0,class1,class2,class3,class4]
scaled_Data = pd.concat(frames)
#finalData.head(10)
print(len(scaled_Data))

In [ ]:
scaled_Data['label'].hist()

# Dataset Splitting

In [ ]:
from sklearn.model_selection import train_test_split

train,valid = train_test_split(scaled_Data, test_size = 0.3, random_state = 42, stratify = scaled_Data.label)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.0)
train_data = train_datagen.flow_from_dataframe(train ,directory='/kaggle/input/cassava-leaf-disease-classification/train_images',
                                                    x_col="image_id" , y_col="label", target_size=(128, 128), color_mode="rgb",
                                                    class_mode="categorical",batch_size=128)
valid_data = train_datagen.flow_from_dataframe(valid ,directory='/kaggle/input/cassava-leaf-disease-classification/train_images',
                                                    x_col="image_id" , y_col="label", target_size=(128, 128), color_mode="rgb",
                                                    class_mode="categorical",batch_size=32)

In [ ]:
import numpy as np
from keras.preprocessing.image import image
from keras.preprocessing.image import img_to_array
#from keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import decode_predictions
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier

import pandas as pd
import io
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator

import json
from tensorflow import keras
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
import random
import tensorflow as tf
import cv2
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from sklearn.model_selection import train_test_split
import re
from functools import partial
import keras
from keras.layers import Dense, Dropout, Input, MaxPooling2D, ZeroPadding2D, Conv2D, Flatten, BatchNormalization
from keras.models import Sequential, Model
from keras.losses import categorical_crossentropy

from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
#from keras.utils import to_categorical
from tensorflow.keras import regularizers

from tensorflow.keras.layers import MaxPool2D, AveragePooling2D, GlobalAveragePooling2D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

from zipfile import ZipFile
import time
from datetime import timedelta
from io import BytesIO

# Image manipulation.
import PIL.Image

import pickle
import os

import random
from keras.callbacks import ReduceLROnPlateau

import matplotlib.pyplot as plt
import numpy as np

# Data Augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal_and_vertical'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(factor=(-0.2,0.2)),
  tf.keras.layers.experimental.preprocessing.RandomCrop(height = 128, width = 128)

])

# Model Building

In [ ]:
from keras.applications.vgg16 import VGG16
image_size=128
def create_vgg():
    inputshape = (image_size, image_size, 3)
    inputs = tf.keras.Input(shape=inputshape)
    x = data_augmentation(inputs)

    base_model = VGG16(include_top = False, input_shape = inputshape, weights="imagenet")

    x = base_model(x)

    model = tf.keras.layers.GlobalAveragePooling2D()(x) 

    model = Dense(256, activation = "relu",kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.L1L2(l1 = 0.01, l2 = 0.01))(model)
    model = Dropout(0.5)(model)

    model = Dense(128,activation='relu',kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.L1L2(l1 = 0.01, l2 = 0.01))(model)
    model = Dropout(0.5)(model)
    out = Dense(5, activation = "softmax")(model)
  
    model = Model(inputs = inputs, outputs = out)
    return model

model_vgg = create_vgg()



In [ ]:
opt = keras.optimizers.Nadam(learning_rate = 1e-5)
model_vgg.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])



# Model Training

In [ ]:
history=model_vgg.fit(train_data,
          epochs = 30,
          validation_data = valid_data,
          callbacks = [es,rlrop,],verbose = 1)

In [ ]:
model_vgg.save("/kaggle/working/resnet50_cassava_final.pkl")

In [ ]:
est_loss, test_acc = model_vgg.evaluate(valid_data, verbose=2) 

print('Test accuracy:', test_acc)

In [ ]:
fig, axs = plt.subplots(nrows = 1, ncols =2 , figsize = (10,5))
axs[0].plot(range(len(history.history['accuracy'])), history.history['accuracy'], linewidth = 2, label = "training")
axs[0].plot(range(len(history.history['accuracy'])), history.history['val_accuracy'], linewidth = 2, label = "validation")
axs[1].plot(range(len(history.history['loss'])), history.history['loss'], linewidth = 2, label = "training")
axs[1].plot(range(len(history.history['loss'])), history.history['val_loss'], linewidth = 2, label = "validation")
axs[0].set_xlabel("epochs")
axs[0].set_xlabel("epochs")
axs[0].set_ylabel("accuracy")
axs[1].set_ylabel("loss")
plt.legend()
plt.savefig("vgg16_plot.pdf")

In [ ]:
from keras.preprocessing import image
test_image = image.load_img('/kaggle/input/cassava-leaf-disease-classification/test_images/2216849948.jpg', 
                            target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model_vgg.predict(test_image)

In [ ]:
submission_data = pd.DataFrame(columns = ['image_id','label'])
submission_data['image_id'] = os.listdir('/kaggle/input/cassava-leaf-disease-classification/test_images')
submission_data['label'] = np.argmax(result)

In [ ]:
submission_data.to_csv('submission.csv',index=False)